In [3]:
from astropy.time import Time
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from lsst_efd_client import EfdClient
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# M1M3 hardpoint repeatability and resolution analysis
---

## Overview
This notebook is dedicated to the analysis of M1M3 hardpoint repeatability.

Data to be analyzed are for the Requirement: "LVV-11200 LTS-88-REQ-0015-V-01: 3.7.1.3 Hardpoint Displacement Repeatability and Resolution_1" - https://jira.lsstcorp.org/browse/LVV-11200

There are three key deliverables
- Plot Force vs. displacement for all actuators.
- Make a histogram of the displacement values per actuator.
- From the generated plot, measure the Hardpoint displacement repeatability (Average) and its resolution (FWHM) around 0 Newtons.

## Querying EFD

### Helper Function

In [4]:
async def query_bump_logs_in_chunks(
    start_date, end_date, client_name="", chunk_size_days=3,topic_name="lsst.sal.MTM1M3.logevent_logMessage",fields=["message"]
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).
        
        end_date (str): End date of the query in ISO format (YYYY-MM-DD).
        
        client_name (str, optional): Name of the EFD client. Defaults to "".
        
        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

def makeClient(client_name):
        # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

## Let's take a look at the topics, so that we can get an idea of what we need to query

In [26]:
client = makeClient("usdf_efd")

a = await client.get_topics() 

In [27]:
for entry in a:
    if entry.__contains__("lsst.sal.MTM1M3"):# and entry.__contains__("isplace"):
        print(entry)

lsst.sal.MTM1M3.accelerometerData
lsst.sal.MTM1M3.ackcmd
lsst.sal.MTM1M3.appliedAccelerationForces
lsst.sal.MTM1M3.appliedAzimuthForces
lsst.sal.MTM1M3.appliedBalanceForces
lsst.sal.MTM1M3.appliedCylinderForces
lsst.sal.MTM1M3.appliedElevationForces
lsst.sal.MTM1M3.appliedForces
lsst.sal.MTM1M3.appliedThermalForces
lsst.sal.MTM1M3.appliedVelocityForces
lsst.sal.MTM1M3.command_abortRaiseM1M3
lsst.sal.MTM1M3.command_applyAberrationForces
lsst.sal.MTM1M3.command_applyActiveOpticForces
lsst.sal.MTM1M3.command_applyActiveOpticForcesByBendingModes
lsst.sal.MTM1M3.command_applyOffsetForces
lsst.sal.MTM1M3.command_applyOffsetForcesByMirrorForce
lsst.sal.MTM1M3.command_boosterValveClose
lsst.sal.MTM1M3.command_boosterValveOpen
lsst.sal.MTM1M3.command_clearActiveOpticForces
lsst.sal.MTM1M3.command_clearOffsetForces
lsst.sal.MTM1M3.command_clearSlewFlag
lsst.sal.MTM1M3.command_disable
lsst.sal.MTM1M3.command_disableForceActuator
lsst.sal.MTM1M3.command_disableHardpointChase
lsst.sal.MTM1M3.comman

### Possibilities for the SAL data

- lsst.sal.MTM1M3.forceActuatorData
- lsst.sal.MTM1M3.forceActuatorPressure
- lsst.sal.MTM1M3.hardpointActuatorData
- lsst.sal.MTM1M3.hardpointMonitorData

Or use chronograf to get a rough idea of the time period of interest